In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import messagebox, filedialog, ttk

# Load the dataset
file_path = 'C:\\Users\\Admin\\Downloads\\predicteddiseases.csv'
data = pd.read_csv(file_path, on_bad_lines='skip')

# Prepare the data
symptoms = data['Symptoms'].tolist()
diseases = data['Predicted Disease'].tolist()

# Vectorize symptoms
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(symptoms)

# Extract unique symptoms for checkboxes
all_symptoms = sorted(set(', '.join(symptoms).split(', ')))

def predict_disease(user_input):
    user_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vector, X).flatten()
    best_match_idx = similarities.argmax()  # Get index of the highest match
    
    return {
        "Predicted Disease": diseases[best_match_idx],
        "Confidence": data['Confidence (%)'].iloc[best_match_idx],
        "Treatment": data['Treatment'].iloc[best_match_idx]
    }

def download_report(name, age, gender, fever, selected_symptoms, prediction):
    report_text = (
        f"Patient Name: {name}\n"
        f"Age: {age}\n"
        f"Gender: {gender}\n"
        f"Fever: {fever} °C\n"
        f"Symptoms: {', '.join(selected_symptoms)}\n"
        f"\nPredicted Disease: {prediction['Predicted Disease']}\n"
        f"Confidence: {prediction['Confidence']:.2f}%\n"
        f"Treatment: {prediction['Treatment']}\n"
    )

    file_path = filedialog.asksaveasfilename(defaultextension=".txt",
                                             filetypes=[("Text files", ".txt"), ("All files", ".*")],
                                             title="Save Report")
    if file_path:
        with open(file_path, "w") as file:
            file.write(report_text)
        messagebox.showinfo("Success", "Report downloaded successfully!")

def show_result_window(prediction, name, age, gender, fever, selected_symptoms):
    result_window = tk.Toplevel(root)
    result_window.title("Prediction Result")
    result_window.geometry("500x450")
    result_window.configure(bg="#FFE4B5")
    
    tk.Label(result_window, text="Prediction Result", font=("Arial", 16, "bold"), fg="blue", bg="#FFE4B5").pack(pady=10)
    tk.Label(result_window, text=f"Name: {name}", font=("Arial", 12, "bold"), bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Age: {age}", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Gender: {gender}", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Fever: {fever} °C", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Symptoms: {', '.join(selected_symptoms)}", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10, pady=10)
    
    tk.Label(result_window, text=f"Disease: {prediction['Predicted Disease']}", font=("Arial", 14, "bold"), fg="red", bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Confidence: {prediction['Confidence']:.2f}%", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10)
    tk.Label(result_window, text=f"Treatment: {prediction['Treatment']}", font=("Arial", 12), bg="#FFE4B5").pack(anchor="w", padx=10)
    
    tk.Button(result_window, text="Download Report", command=lambda: download_report(name, age, gender, fever, selected_symptoms, prediction), 
              font=("Arial", 12), bg="#008CBA", fg="white", width=20).pack(pady=5)
    
    tk.Button(result_window, text="Close", command=result_window.destroy, font=("Arial", 12), bg="#FF4500", fg="white", width=20).pack(pady=5)

def run_prediction():
    name = name_entry.get().strip()
    age = age_entry.get().strip()
    gender = gender_var.get()
    fever = fever_entry.get().strip()
    
    if not name or not age or not fever or gender == "Select":
        messagebox.showwarning("Input Error", "Please fill in all details.")
        return
    
    selected_symptoms = [symptom for symptom, var in checkbox_vars.items() if var.get() == 1]
    
    if not selected_symptoms:
        messagebox.showwarning("Input Error", "Please select at least one symptom.")
        return
    
    user_input = ', '.join(selected_symptoms)
    prediction = predict_disease(user_input)
    show_result_window(prediction, name, age, gender, fever, selected_symptoms)

root = tk.Tk()
root.title("Disease Prediction System")
root.geometry("600x700")
root.configure(bg="#ADD8E6")

tk.Label(root, text="Disease Prediction System", font=("Arial", 18, "bold"), fg="hotpink", bg="#ADD8E6").pack(pady=10)

details_frame = tk.Frame(root, bg="#ADD8E6")
details_frame.pack(pady=10)

fields = ["Name", "Age", "Gender", "Fever (°C)"]
entries = {}
for idx, field in enumerate(fields):
    tk.Label(details_frame, text=field+":", font=("Arial", 12), bg="#ADD8E6").grid(row=idx, column=0, padx=5, pady=5)
    if field == "Gender":
        gender_var = tk.StringVar(value="Select")
        gender_menu = tk.OptionMenu(details_frame, gender_var, "Male", "Female", "Other")
        gender_menu.grid(row=idx, column=1)
    else:
        entries[field] = tk.Entry(details_frame, width=20, font=("Arial", 12))
        entries[field].grid(row=idx, column=1)
name_entry, age_entry, fever_entry = entries["Name"], entries["Age"], entries["Fever (°C)"]

symptoms_frame = tk.Frame(root, bg="#ADD8E6")
symptoms_frame.pack(pady=10, fill="both", expand=True)

tk.Label(symptoms_frame, text="Select Symptoms:", font=("Arial", 12, "bold"), bg="#ADD8E6").pack()

canvas = tk.Canvas(symptoms_frame, bg="#ADD8E6", height=200)
scrollbar = ttk.Scrollbar(symptoms_frame, orient="vertical", command=canvas.yview)
scrollable_frame = tk.Frame(canvas, bg="#ADD8E6")

scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(
        scrollregion=canvas.bbox("all")
    )
)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

checkbox_vars = {}
for symptom in all_symptoms:
    var = tk.IntVar()
    tk.Checkbutton(scrollable_frame, text=symptom, variable=var, font=("Arial", 10), fg="green", bg="#ADD8E6", anchor="w").pack(anchor="w", padx=5)
    checkbox_vars[symptom] = var

canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

tk.Button(root, text="Predict Disease", command=run_prediction, font=("Arial", 12, "bold"), bg="#4CAF50", fg="white", width=15).pack(pady=10)

root.mainloop()